In [1]:
import sys   ###qmdp1（上の2つのセル）
sys.path.append('../scripts/')
from dp_policy_agent import *

In [2]:
class QmdpAgent(DpPolicyAgent):
    def __init__(self, time_interval, estimator, goal, puddles, sampling_num=10, widths=np.array([0.2, 0.2, math.pi/18]).T, \
                 puddle_coef=100.0, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T): 
        super().__init__(time_interval, estimator, goal, puddle_coef, widths, lowerleft, upperright)
        
    def normalize1(self, t): #正規化方法1（-π〜π）
            while t < -np.pi: t += 2*np.pi
            while t >= np.pi: t -= 2*np.pi
            return t
        
    def normalize2(self, t): #正規化方法1（0〜2π）
            while t < 0.0: t += 2*np.pi
            while t >= 2*np.pi: t -= 2*np.pi
            return t
        
    def avg(self, particles):
        x = np.array([p.pose[0] for p in particles]).mean()
        y = np.array([p.pose[1] for p in particles]).mean()
        
        ts1 = np.array([self.normalize1(p.pose[2]) for p in particles])
        ts2 = np.array([self.normalize2(p.pose[2]) for p in particles])
        
        t = ts1.mean() if ts1.var() < ts2.var() else ts2.mean()
        
        return np.array([x,y,t]).T
        
    def policy(self, pose, goal=None): #平均値を返す
        pose = self.avg(self.estimator.particles)
        return self.policy_data[self.to_index(pose, self.pose_min, self.index_nums, self.widths)]

In [3]:
def trial(animation):  ###qmdp1exec
    time_interval = 0.1
    world = PuddleWorld(30, time_interval, debug=not animation) 

    ##ランドマークの追加（意地悪な位置に）##
    m = Map()
    for ln in [(1,4), (4,1), (-4,-4)]: m.append_landmark(Landmark(*ln))
    world.append(m) 
    
    ##ゴール・水たまりの追加（これは特に変更なし）##
    goal = Goal(-3,-3)
    puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] 
    world.append(goal)
    world.append(puddles[0]) 
    world.append(puddles[1])
    
    ##ロボットを作る##
    init_pose = np.array([2.5, 2.5, 0]).T
    pf = Mcl(m, init_pose, 100)
    a = QmdpAgent(time_interval, pf, goal, puddles)
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")

    world.append(r)
        
    world.draw()
    
    return a

In [ ]:
def evaluation():
    with open("ml_result.txt", "w") as f:
        num = 1000
        for i in range(num):
            a = trial(False)
            f.write("{} {}\n".format(a.total_reward+a.final_value, a.in_goal))
            f.flush()
            
evaluation()

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
